## Laptop price

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import *

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"julsfeo","key":"129b619f1cecf1369046e2c1c24d486b"}'}

In [3]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download -c laptop-price

  0% 0.00/29.6k [00:00<?, ?B/s]
100% 29.6k/29.6k [00:00<00:00, 33.6MB/s]


In [5]:
! unzip laptop-price.zip -d laptop-price

Archive:  laptop-price.zip
  inflating: laptop-price/Price_euros_test.csv  
  inflating: laptop-price/Price_euros_train.csv  
  inflating: laptop-price/Sample_submition.csv  


In [43]:
df_train = pd.read_csv('/content/laptop-price/Price_euros_test.csv')

df_train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,86,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,1TB HDD,AMD Radeon R5,Windows 10,2.1kg
1,1249,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,1TB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg
2,122,Asus,VivoBook S15,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.7kg
3,993,Asus,Rog GL753VE-DS74,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99kg
4,739,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.25kg


In [44]:
df_train.dtypes

,0
laptop_ID,int64
Company,object
Product,object
TypeName,object
Inches,float64
ScreenResolution,object
Cpu,object
Ram,object
Memory,object
Gpu,object


In [45]:
df_train.isnull().sum()

,0
laptop_ID,0
Company,0
Product,0
TypeName,0
Inches,0
ScreenResolution,2
Cpu,0
Ram,0
Memory,0
Gpu,0


In [46]:
df_train_nan = df_train.loc[:, df_train.isnull().any()]
df_train_nan.isnull().sum() / len(df_train_nan) * 100

,0
ScreenResolution,1.025641
OpSys,6.153846
Weight,2.051282


In [47]:
df_train['OpSys'].value_counts()

,count
OpSys,
Windows 10,156
Windows 7,8
Linux,7
No OS,5
Chrome OS,2
Windows 10 S,2
macOS,2
Mac OS X,1


In [48]:
df_train['Memory'].value_counts()

,count
Memory,
256GB SSD,68
1TB HDD,36
512GB SSD,18
500GB HDD,14
128GB SSD + 1TB HDD,13
128GB SSD,12
256GB SSD + 1TB HDD,7
32GB Flash Storage,4
1TB SSD,3


In [49]:
df_train['OpSys'] = df_train['OpSys'].replace({'Windows 10': 9, 'No OS': 8, 'Linux': 7, 'Windows 7' : 6, 'Chrome OS': 5,
                                   'macOS': 4, 'Mac OS X': 3, 'Windows 10 S': 2, 'Android': 1})
df_train['OpSys'] = df_train['OpSys'].fillna(0)

<ipython-input-49-220f1dec4602>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['OpSys'] = df_train['OpSys'].replace({'Windows 10': 9, 'No OS': 8, 'Linux': 7, 'Windows 7' : 6, 'Chrome OS': 5,


In [50]:
df_train['Weight'].value_counts()

,count
Weight,
2.2kg,11
2.3kg,8
2.5kg,8
2.1kg,7
1.4kg,7
...,...
2.83kg,1
1.54kg,1
1.29kg,1


In [51]:
df_train['Weight'] = df_train['Weight'].fillna(df_train['Weight'].mode().iloc[0])

In [52]:
df_train['ScreenResolution'] = df_train['ScreenResolution'].fillna(df_train['ScreenResolution'].mode().iloc[0])

In [53]:
df_train[['MainMemory', 'AddMemory']] = df_train['Memory'].str.split('+', expand=True)

df_train['MainMemory'] = df_train['MainMemory'].str.strip()
df_train['AddMemory'] = df_train['AddMemory'].str.strip()

df_train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,MainMemory,AddMemory
0,86,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,1TB HDD,AMD Radeon R5,9.0,2.1kg,1TB HDD,None
1,1249,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,1TB SSD,Nvidia GeForce GTX 1060,9.0,1.95kg,1TB SSD,None
2,122,Asus,VivoBook S15,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Nvidia GeForce 940MX,9.0,1.7kg,256GB SSD,None
3,993,Asus,Rog GL753VE-DS74,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,9.0,2.99kg,256GB SSD,1TB HDD
4,739,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,1TB HDD,Intel HD Graphics 620,9.0,2.25kg,1TB HDD,None


In [54]:
df_train['AddMemory'].value_counts()

,count
AddMemory,
1TB HDD,23
2TB HDD,1
1.0TB Hybrid,1
256GB SSD,1


In [55]:
df_train['AddMemory'] = df_train['AddMemory'].fillna(0)

In [56]:
df_train['Ram'] = df_train['Ram'].astype(str)
df_train['Ram'] = df_train['Ram'].str.extract(r'(\d+)')
df_train['Ram'] = df_train['Ram'].astype(int)

df_train['Weight'] = df_train['Weight'].str.extract(r'(\d+\.\d+)')
df_train['Weight'] = df_train['Weight'].astype(float)

cpu_pattern = r'(.+?)\s+(\d+\.?\d*)\s*GHz'
extracted_cpu = df_train['Cpu'].str.extract(cpu_pattern)
df_train[['Cpu_Model', 'Cpu_Frequency']] = extracted_cpu
df_train['Cpu_Frequency'] = df_train['Cpu_Frequency'].astype(float)

screen_pattern = r'(?:(Full HD|IPS Panel|Touchscreen|4K Ultra HD|Retina Display|HD)?\s*(\d+)\s*x\s*(\d+))'
df_train[['Screen_Type', 'Width', 'Height']] = df_train['ScreenResolution'].str.extract(screen_pattern)
df_train['Width'] = df_train['Width'].astype(int)
df_train['Height'] = df_train['Height'].astype(int)

In [57]:
memory_pattern = r'(\d+\.?\d*\s*(GB|TB))\s*(HDD|SSD|Flash Storage|Hybrid)?'
df_train[['MainMemory_Amount', 'MainMemory_Unit', 'MainMemory_Type']] = df_train['MainMemory'].str.extract(memory_pattern)
df_train['MainMemory_Amount'] = df_train['MainMemory_Amount'].str.replace('GB', '', regex = False)
df_train['MainMemory_Amount'] = df_train['MainMemory_Amount'].str.replace('TB', '', regex = False)
df_train['MainMemory_Amount'] = df_train['MainMemory_Amount'].astype(float)

df_train[['AddMemory_Amount', 'AddMemory_Unit', 'AddMemory_Type']] = df_train['AddMemory'].str.extract(memory_pattern)
df_train['AddMemory_Amount'] = df_train['AddMemory_Amount'].str.replace('GB', '', regex = False)
df_train['AddMemory_Amount'] = df_train['AddMemory_Amount'].str.replace('TB', '', regex = False)
df_train['AddMemory_Amount'] = df_train['AddMemory_Amount'].astype(float)

In [58]:
df_train = df_train.drop('Cpu', axis = 1)
df_train = df_train.drop('ScreenResolution', axis = 1)
df_train['AddMemory_Amount'] = df_train['AddMemory_Amount'].replace({1.0 : 1024.0, 1: 1024.0, 2: 2048.0})
df_train['MainMemory_Amount'] = df_train['MainMemory_Amount'].replace({1.0 : 1024.0, 1: 1024.0, 2: 2048.0})
df_train = df_train.drop('AddMemory_Unit', axis = 1)
df_train = df_train.drop('MainMemory_Unit', axis = 1)
df_train = df_train.drop('AddMemory', axis = 1)
df_train = df_train.drop('MainMemory', axis = 1)
df_train = df_train.drop('Screen_Type', axis = 1)

In [59]:
df_train.head()

,laptop_ID,Company,Product,TypeName,Inches,Ram,Memory,Gpu,OpSys,Weight,Cpu_Model,Cpu_Frequency,Width,Height,MainMemory_Amount,MainMemory_Type,AddMemory_Amount,AddMemory_Type
0,86,Acer,Aspire 3,Notebook,15.6,4,1TB HDD,AMD Radeon R5,9.0,2.10,AMD A9-Series 9420,3.0,1366,768,1024.0,HDD,NaN,NaN
1,1249,Razer,Blade Pro,Gaming,14.0,16,1TB SSD,Nvidia GeForce GTX 1060,9.0,1.95,Intel Core i7 7700HQ,2.8,1920,1080,1024.0,SSD,NaN,NaN
2,122,Asus,VivoBook S15,Notebook,15.6,8,256GB SSD,Nvidia GeForce 940MX,9.0,1.70,Intel Core i7 8550U,1.8,1920,1080,256.0,SSD,NaN,NaN
3,993,Asus,Rog GL753VE-DS74,Gaming,17.3,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,9.0,2.99,Intel Core i7 7700HQ,2.8,1920,1080,256.0,SSD,1024.0,HDD
4,739,Dell,Inspiron 3567,Notebook,15.6,12,1TB HDD,Intel HD Graphics 620,9.0,2.25,Intel Core i5 7200U,2.5,1366,768,1024.0,HDD,NaN,NaN


In [60]:
df_train['AddMemory_Amount'].value_counts()

,count
AddMemory_Amount,
1024.0,24
2048.0,1
256.0,1


In [61]:
df_train['MainMemory_Type'].value_counts()

,count
MainMemory_Type,
SSD,131
HDD,52
Flash Storage,10
Hybrid,2


In [62]:
df_train['MainMemory_Type'] = df_train['MainMemory_Type'].replace({'SSD': 4, 'HDD': 3, 'Flash Storage': 2, 'Hybrid' : 1})
df_train['AddMemory_Type'] = df_train['AddMemory_Type'].replace({'SSD': 2, 'HDD': 3, 'Hybrid' : 1})

<ipython-input-62-f69fe696a097>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['MainMemory_Type'] = df_train['MainMemory_Type'].replace({'SSD': 4, 'HDD': 3, 'Flash Storage': 2, 'Hybrid' : 1})
<ipython-input-62-f69fe696a097>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['AddMemory_Type'] = df_train['AddMemory_Type'].replace({'SSD': 2, 'HDD': 3, 'Hybrid' : 1})


In [63]:
df_train = df_train.drop('Memory', axis = 1)

In [64]:
df_train.isnull().sum()

,0
laptop_ID,0
Company,0
Product,0
TypeName,0
Inches,0
Ram,0
Gpu,0
OpSys,0
Weight,5
Cpu_Model,0


In [65]:
df_train['Weight'] = df_train['Weight'].fillna(df_train['Weight'].mode().iloc[0])
df_train['AddMemory_Amount'] = df_train['AddMemory_Amount'].fillna(0)
df_train['AddMemory_Type'] = df_train['AddMemory_Type'].fillna(0)

In [66]:
df_train['Company'].unique().tolist()

['Acer',
 'Razer',
 'Asus',
 'Dell',
 'Lenovo',
 'Huawei',
 'Mediacom',
 'HP',
 'MSI',
 'Toshiba',
 'Apple',
 'LG',
 'Microsoft',
 'Chuwi']

In [67]:
df_train['Company'].value_counts()

,count
Company,
Lenovo,46
Dell,42
HP,42
Asus,23
Acer,12
MSI,10
Toshiba,7
Apple,4
Razer,3


In [68]:
df_train['Company'] = df_train['Company'].replace({'Apple': 11,
 'Acer': 14,
 'Asus': 15,
 'HP': 16,
 'Dell': 18,
 'Lenovo': 17,
 'MSI': 13,
 'Microsoft': 8,
 'Toshiba': 12,
 'Xiaomi': 6,
 'Vero': 5,
 'Chuwi': 2,
 'Mediacom': 9,
 'Samsung' : 10,
 'Google': 4,
 'Fujitsu': 3,
 'Razer': 7,
 'LG': 1,
'Huawei': 0})

<ipython-input-68-5b1eb2fa1810>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['Company'] = df_train['Company'].replace({'Apple': 11,


In [69]:
df_train['Cpu_Model'].value_counts()

,count
Cpu_Model,
Intel Core i5 7200U,33
Intel Core i7 7700HQ,24
Intel Core i7 7500U,18
Intel Core i5 6200U,12
Intel Core i3 6006U,11
Intel Core i7 8550U,10
Intel Core i5 8250U,9
Intel Core i7 6700HQ,8
Intel Core i7 6500U,7


In [70]:
def group_processors(processor):
    if 'Intel Core i5' in processor:
        return 'Intel Core i5'
    elif 'Intel Core i7' in processor:
        return 'Intel Core i7'
    elif 'Intel Core i3' in processor:
        return 'Intel Core i3'
    elif 'Intel Core M' in processor:
        return 'Intel Core M'
    elif 'Intel Xeon' in processor:
        return 'Intel Xeon'
    elif 'Intel Celeron' in processor:
        return 'Intel Celeron'
    elif 'Intel Pentium' in processor:
        return 'Intel Pentium'
    elif 'Intel Atom' in processor:
        return 'Intel Atom'
    elif 'AMD Ryzen' in processor:
        return 'AMD Ryzen'
    elif 'AMD A' in processor:
        return 'AMD A-Series'
    elif 'AMD E' in processor:
        return 'AMD E-Series'
    elif 'AMD FX' in processor:
        return 'AMD FX'
    elif 'Samsung' in processor:
        return 'Samsung'
    else:
        return 'Other'

df_train['Cpu_Model'] = df_train['Cpu_Model'].apply(group_processors)

processor_group_map = {
    'Intel Core i7': 14,
    'Intel Core i5': 13,
    'Intel Core i3': 12,
    'Intel Core M': 11,
    'Intel Xeon': 10,
    'Intel Celeron': 9,
    'Intel Pentium': 8,
    'Intel Atom': 7,
    'AMD Ryzen': 6,
    'AMD A-Series': 5,
    'AMD E-Series': 4,
    'AMD FX': 3,
    'Samsung': 2,
    'Other': 1
}

df_train['Cpu_Model'] = df_train['Cpu_Model'].replace(processor_group_map)

<ipython-input-70-37278559a9b0>:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['Cpu_Model'] = df_train['Cpu_Model'].replace(processor_group_map)


In [71]:
df_train['Cpu_Model'].value_counts()

,count
Cpu_Model,
14,84
13,65
12,15
9,13
5,9
8,7
11,1
7,1


In [72]:
df_train.dtypes

,0
laptop_ID,int64
Company,int64
Product,object
TypeName,object
Inches,float64
Ram,int64
Gpu,object
OpSys,float64
Weight,float64
Cpu_Model,int64


In [73]:
df_train['TypeName'].value_counts()

,count
TypeName,
Notebook,100
Gaming,35
Ultrabook,30
2 in 1 Convertible,21
Netbook,5
Workstation,4


In [74]:
df_train['TypeName'] = df_train['TypeName'].replace({'Workstation': 2, '2 in 1 Convertible': 3, 'Netbook' : 1, 'Ultrabook': 4, 'Gaming' : 5, 'Notebook' : 6})

<ipython-input-74-52c9e1804cb1>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['TypeName'] = df_train['TypeName'].replace({'Workstation': 2, '2 in 1 Convertible': 3, 'Netbook' : 1, 'Ultrabook': 4, 'Gaming' : 5, 'Notebook' : 6})


In [75]:
df_train['Gpu'].unique().tolist()

['AMD Radeon R5',
 'Nvidia GeForce GTX 1060',
 'Nvidia GeForce 940MX',
 'Nvidia GeForce GTX 1050 Ti',
 'Intel HD Graphics 620',
 'Nvidia Quadro M2200M',
 'Intel HD Graphics 500',
 'Nvidia GeForce GTX 960M',
 'Nvidia GeForce GTX 1050',
 'Intel HD Graphics 520',
 'Intel UHD Graphics 620',
 'Nvidia GeForce GTX 1070',
 'Nvidia GeForce 930MX',
 'Nvidia GeForce 930M',
 'AMD Radeon R7 M445',
 'AMD Radeon R4',
 'Nvidia GeForce GTX 1080',
 'Intel HD Graphics 405',
 'Nvidia GeForce GTX 960',
 'Intel HD Graphics 615',
 'Intel HD Graphics 400',
 'Nvidia Quadro M620',
 'Intel HD Graphics',
 'Intel HD Graphics 630',
 'Nvidia GeForce GTX 1050Ti',
 'AMD Radeon R7 M440',
 'Nvidia GeForce GTX 970M',
 'AMD Radeon R5 M430',
 'Intel HD Graphics 505',
 'AMD FirePro W4190M',
 'AMD Radeon R5 M330',
 'Nvidia GeForce GTX 965M',
 'AMD Radeon R5 430',
 'Nvidia GeForce 920M',
 'Intel HD Graphics 515',
 'Intel Iris Plus Graphics 640',
 'Nvidia GeForce GTX 960<U+039C>',
 'AMD Radeon R7 M460',
 'Nvidia Quadro M2200',

In [76]:
def group_by_company(gpu):
    if 'Intel' in gpu:
        return 'Intel'
    elif 'AMD' in gpu:
        return 'AMD'
    elif 'Nvidia' in gpu:
        return 'Nvidia'
    elif 'ARM' in gpu:
        return 'ARM'
    else:
        return 'Other'

df_train['Gpu'] = df_train['Gpu'].apply(group_by_company)
company_map = {
    'Intel': 5,
    'AMD': 3,
    'Nvidia': 4,
    'ARM': 2,
    'Other': 1
}

# Заменяем группы на числовые значения
df_train['Gpu'] = df_train['Gpu'].replace(company_map)

<ipython-input-76-a95782690a1f>:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['Gpu'] = df_train['Gpu'].replace(company_map)


In [77]:
df_train.head()

,laptop_ID,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Cpu_Model,Cpu_Frequency,Width,Height,MainMemory_Amount,MainMemory_Type,AddMemory_Amount,AddMemory_Type
0,86,14,Aspire 3,6,15.6,4,3,9.0,2.10,5,3.0,1366,768,1024.0,3,0.0,0.0
1,1249,7,Blade Pro,5,14.0,16,4,9.0,1.95,14,2.8,1920,1080,1024.0,4,0.0,0.0
2,122,15,VivoBook S15,6,15.6,8,4,9.0,1.70,14,1.8,1920,1080,256.0,4,0.0,0.0
3,993,15,Rog GL753VE-DS74,5,17.3,16,4,9.0,2.99,14,2.8,1920,1080,256.0,4,1024.0,3.0
4,739,18,Inspiron 3567,6,15.6,12,5,9.0,2.25,13,2.5,1366,768,1024.0,3,0.0,0.0


In [42]:
df_train.to_csv('preprocessed_train.csv', index=False)

In [78]:
df_train.to_csv('preprocessed_test.csv', index=False)

In [79]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

In [80]:
df = pd.read_csv('preprocessed_train.csv')
X_train = df.drop(columns=['laptop_ID', 'Price_euros', 'Product'])
Y = df['Price_euros']

std_X = StandardScaler()
std_Y = StandardScaler()

X_std_train = std_X.fit_transform(X_train)
Y_std = std_Y.fit_transform(Y.values.reshape(-1, 1)).flatten()

X_train_tensor = torch.tensor(X_std_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_std, dtype=torch.float32)


In [81]:
df_test = pd.read_csv('preprocessed_test.csv')
X_test = df_test.drop(columns=['laptop_ID', 'Product'])
X_std_test = std_X.transform(X_test)

X_test_tensor = torch.tensor(X_std_test, dtype=torch.float32)

In [82]:
class LaptopModel(nn.Module):
    def __init__(self, input_size, n_hidden_neurons):
        super(LaptopModel, self).__init__()
        self.fc1 = nn.Linear(input_size, n_hidden_neurons)
        self.act1 = nn.Sigmoid()
        self.fc2 = nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.act2 = nn.Sigmoid()
        self.fc3 = nn.Linear(n_hidden_neurons, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x

model = LaptopModel(input_size=X_train.shape[1], n_hidden_neurons=64)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, Y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 0.3248
Epoch [200/1000], Loss: 0.2754
Epoch [300/1000], Loss: 0.2612
Epoch [400/1000], Loss: 0.2461
Epoch [500/1000], Loss: 0.2255
Epoch [600/1000], Loss: 0.2037
Epoch [700/1000], Loss: 0.1822
Epoch [800/1000], Loss: 0.1634
Epoch [900/1000], Loss: 0.1464
Epoch [1000/1000], Loss: 0.1309


In [83]:
model.eval()
with torch.no_grad():
    predictions_scaled = model(X_test_tensor).numpy().flatten()

predictions = std_Y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()

submission = pd.DataFrame({
    "index": range(len(predictions)),
    "Price_euros": predictions
})

submission.to_csv("submission.csv", index=False)
print("Файл submission.csv сохранен!")

Файл submission.csv сохранен!
